In [2]:
import pandas as pd
import numpy as np

In [3]:
expeditions = pd.read_csv('./data/expeditions.csv')
members = pd.read_csv('./data/members.csv')
peaks = pd.read_csv('./data/peaks.csv')

### Question 1

#### A) Get proportion of unclimbed peaks

In [4]:
# Get proportion of peaks unclimbed
unclimbed_one = (peaks[peaks['climbing_status'] == 'Unclimbed'].value_counts('climbing_status')/len(peaks))
unclimbed_one # around 27% are still unclimbed


climbing_status
Unclimbed    0.271368
dtype: float64

In [5]:
#OR

unclimbed_two = (peaks['climbing_status'].value_counts()['Unclimbed']/len(peaks))
unclimbed_two

0.27136752136752135

#### B) Average Height of Climbed & Unclimbed Peaks

In [6]:
avg_height = peaks.groupby('climbing_status')['height_metres'].mean()
avg_height # 6706.3 for Climbed, 6523.3 for Unclimbed

climbing_status
Climbed      6706.284457
Unclimbed    6523.330709
Name: height_metres, dtype: float64

### Question 2

#### A) Find Percentage of Sherpas Who Were Hired

In [7]:
members_citizenship = members[members['citizenship'] == 'Nepal']
members_citizenship # get Sherpas

,expedition_id,member_id,peak_id,peak_name,year,season,sex,age,citizenship,expedition_role,...,highpoint_metres,success,solo,oxygen_used,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
24,AMAD79101,AMAD79101-18,AMAD,Ama Dablam,1979,Spring,M,23.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
48,AMAD79301,AMAD79301-26,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
49,AMAD79301,AMAD79301-24,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
50,AMAD79301,AMAD79301-25,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
82,AMAD81101,AMAD81101-11,AMAD,Ama Dablam,1981,Spring,M,25.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76506,PUMO19102,PUMO19102-01,PUMO,Pumori,2019,Spring,M,35.0,Nepal,Leader,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76507,PUMO19102,PUMO19102-02,PUMO,Pumori,2019,Spring,M,52.0,Nepal,H-A Worker,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76513,PUTH19101,PUTH19101-06,PUTH,Putha Hiunchuli,2019,Spring,M,NaN,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76514,PUTH19101,PUTH19101-07,PUTH,Putha Hiunchuli,2019,Spring,M,44.0,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN


In [8]:
members_cit_hired = members_citizenship['hired'].value_counts()[True]/len(members_citizenship)
members_cit_hired # hired percentage ~ 93.4%

0.9340563991323211

#### B) Min, Max, Mean Age of Sherpas

In [9]:
members_cit_age = members_citizenship[['age','citizenship']].groupby('citizenship').agg(['min', 'max', 'mean'])
members_cit_age

age                 
              min   max       mean
citizenship                       
Nepal        12.0  85.0  32.885062

### Question 3

FIX MERGES, SOMETHING IS OFF ABOUT THEM!!!!

In [10]:
merge_1_cols = peaks.columns.intersection(expeditions.columns)
merge_2_cols = expeditions.columns.intersection(members.columns)

print(merge_1_cols)
print(merge_2_cols)


Index(['peak_id', 'peak_name'], dtype='object')
Index(['expedition_id', 'peak_id', 'peak_name', 'year', 'season',
       'highpoint_metres', 'oxygen_used'],
      dtype='object')


In [23]:
merged_data = pd.merge(pd.merge(peaks, expeditions, 
         on = 'peak_id'), members[members['hired'] == False],
         on = 'expedition_id', indicator = True)

merged_data.duplicated()

False    60782
dtype: int64

In [50]:
first_female = merged_data[(
    merged_data['sex'] == 'F') & (merged_data['success'] == True)].nsmallest(
    1, 'year_x', keep = 'all')[['peak_name', 'height_metres', 'year_x']]

first_female

,peak_name,height_metres,year_x
57800,Tengkoma,6215,1949
58228,Dzanye,6581,1949


In [16]:
(merged_data[['success', 'oxygen_used_y', 'died', 'sex']]
.groupby('sex')
.apply(lambda x: x.sum()/len(x))
)

,success,oxygen_used_y,died
sex,,,
F,0.326583,0.215773,0.006988
M,0.312621,0.174955,0.013744


In [18]:
#run checks

members_test = members[['success', 'oxygen_used', 'died', 'sex', 'hired']]
members_test = members_test[(members_test['hired'] == False) & (members_test['sex'] == 'F')]
members_test

,success,oxygen_used,died,sex,hired
25,False,False,False,F,False
28,False,False,False,F,False
37,False,False,False,F,False
46,False,False,False,F,False
93,False,False,False,F,False
...,...,...,...,...,...
76455,True,True,False,F,False
76458,True,False,False,F,False
76464,True,True,False,F,False
76467,True,True,False,F,False


In [20]:
members_test[['success', 'died', 'oxygen_used']].apply(lambda x: x.sum()/len(x))

success        0.326583
died           0.006988
oxygen_used    0.215773
dtype: float64

### Question 4

#### Count Number of Expeditions, Members, and Death Rates for Hired/Unhired

In [66]:
expeditions['decade'] = ((expeditions['year']//10)*10)
expeditions_grouped = (expeditions[['decade','expedition_id', 'members', 'member_deaths', 'hired_staff', 'hired_staff_deaths']]
                       .drop_duplicates(subset = ['expedition_id'])
                       .groupby('decade'))

In [67]:
exped_totals = expeditions_grouped[['expedition_id', 'members']].agg({
   'expedition_id' : 'count', 
   'members' : 'sum'}).rename(
      columns = {'expedition_id' : 'Expeditions', 'members' : 'Members'})
      
exped_totals

,Expeditions,Members
decade,,
1900,4,9
1910,3,3
1920,10,51
1930,34,152
1940,15,53
1950,168,856
1960,161,989
1970,361,3506
1980,1187,10023


In [68]:
death_rate1 = (100*(expeditions_grouped['member_deaths'].sum())/(expeditions_grouped['members'].sum()))
death_rate2 = (100*(expeditions_grouped['hired_staff_deaths'].sum())/(expeditions_grouped['hired_staff'].sum()))
death_rates = pd.concat([death_rate1, death_rate2], axis= 1, keys = ['Member_Deaths', 'Hired_Deaths'])

In [69]:
summarized_exped = pd.merge(exped_totals, death_rates, on = 'decade')

summarized_exped

,Expeditions,Members,Member_Deaths,Hired_Deaths
decade,,,,
1900,4,9,11.111111,100.000000
1910,3,3,0.000000,NaN
1920,10,51,7.843137,8.403361
1930,34,152,1.315789,1.465201
1940,15,53,0.000000,0.000000
1950,168,856,1.869159,1.196809
1960,161,989,2.224469,1.036269
1970,361,3506,2.766686,2.655678
1980,1187,10023,1.995411,1.537958


#### Key Findings:

1. Expeditions have increased *a lot* in the last 4 decades.
2. Anyone hired for an expedition in the 1900s died, which is pretty absurd, and likely a sign of how expedition leaders treated their guides. There were so few expeditions with so few people it makes sense the percentage could be so high. 
3. The death rates spiked in the 60s and 70s, along with the number of expeditions. Since then, the death rates appear to have gone down each decade, despite more people attempting expeditions. 

In [71]:
begin = pd.to_datetime(expeditions['basecamp_date'])
end = pd.to_datetime(expeditions['termination_date'])
dates = pd.concat([begin, end, expeditions['peak_id']], axis = 1).dropna()
dates['length_of_expedition'] = end-begin

dates['peak_length_score'] = dates[['peak_id', 'length_of_expedition']].groupby(
                                                        'peak_id'
                                                             ).transform(lambda x: (x - x.mean()) / x.std())

dates


,basecamp_date,termination_date,peak_id,length_of_expedition,peak_length_score
1,1969-09-25,1969-10-26,ANN2,31 days,-0.074159
3,1978-09-08,1978-10-05,ANN2,27 days,-0.400456
5,1980-03-25,1980-05-01,ANN2,37 days,0.415288
8,1981-09-04,1981-10-17,ANN2,43 days,0.904734
9,1982-09-14,1982-10-08,ANN2,24 days,-0.645179
...,...,...,...,...,...
10359,2019-04-04,2019-05-27,PUMO,53 days,4.141132
10360,2019-04-14,2019-05-25,PUMO,41 days,2.813516
10361,2019-05-09,2019-05-17,PUTH,8 days,-0.925485
10362,2019-04-11,2019-05-03,RATC,22 days,0.653566


In [35]:
length_averages = dates[['peak_id', 'length_of_expedition']]
length_averages.groupby('peak_id').mean().sort_values('length_of_expedition')

,length_of_expedition
peak_id,
PARC,0 days 00:00:00
SHAL,1 days 00:00:00
MARD,1 days 00:00:00
PBUK,2 days 00:00:00
CHRI,2 days 00:00:00
...,...
KHUM,54 days 00:00:00
LHOM,60 days 00:00:00
DZAN,60 days 00:00:00


In [37]:
deaths = expeditions[['peak_id','member_deaths', 'hired_staff_deaths']].groupby('peak_id').agg('mean')

deaths_vs_length = pd.merge(deaths, length_averages, on = 'peak_id')

deaths_vs_length

,peak_id,member_deaths,hired_staff_deaths,length_of_expedition
0,AMAD,0.019034,0.004392,25 days
1,AMAD,0.019034,0.004392,11 days
2,AMAD,0.019034,0.004392,18 days
3,AMAD,0.019034,0.004392,18 days
4,AMAD,0.019034,0.004392,12 days
...,...,...,...,...
7941,YANS,0.000000,0.000000,10 days
7942,YANS,0.000000,0.000000,19 days
7943,YAUP,0.000000,0.000000,54 days
7944,YAUP,0.000000,0.000000,3 days


Now lets see if expedition length is associated with deaths in anyway. Doing a basic sort should paint the picture:

In [62]:
deaths_vs_length.sort_values(['hired_staff_deaths', 'length_of_expedition'])

,peak_id,member_deaths,hired_staff_deaths,length_of_expedition
7282,PARC,0.000000,0.000000,0 days
7675,SARI,0.000000,0.000000,0 days
7780,TILI,0.104651,0.000000,0 days
7848,TKPO,0.111111,0.000000,0 days
1426,ARNK,0.000000,0.000000,1 days
...,...,...,...,...
5209,GURJ,1.000000,0.444444,19 days
5210,GURJ,1.000000,0.444444,22 days
5211,GURJ,1.000000,0.444444,35 days
3231,DHA4,0.818182,0.454545,50 days


Looking at it this way, not really seeing anything substantial here when it comes to longer expeditions and more deaths. I think generally you'll see lower deaths with shorter expeditions, which is intuitive. Maybe if we group by peak we'll see something:

In [61]:
association_check = deaths_vs_length.groupby('peak_id').agg(
    'mean', numeric_only = False)
    
association_check.nlargest(20, 'hired_staff_deaths')


,member_deaths,hired_staff_deaths,length_of_expedition
peak_id,,,
DHA4,0.818182,0.454545,52 days 12:00:00
GURJ,1.000000,0.444444,20 days 14:24:00
KGUR,0.258065,0.354839,13 days 11:25:42.857142857
JETH,0.000000,0.333333,36 days 00:00:00
ANNM,0.142857,0.285714,28 days 00:00:00
GANG,0.185185,0.222222,27 days 10:06:18.947368421
LEON,0.111111,0.222222,36 days 09:36:00
NAG1,0.000000,0.200000,18 days 14:24:00
DHA2,0.062500,0.187500,34 days 13:20:00


In [60]:
association_check.nlargest(20, 'member_deaths')

,member_deaths,hired_staff_deaths,length_of_expedition
peak_id,,,
GURJ,1.000000,0.444444,20 days 14:24:00
HIMN,1.000000,0.000000,28 days 12:00:00
DHA4,0.818182,0.454545,52 days 12:00:00
GAN5,0.800000,0.000000,28 days 18:00:00
DHA5,0.666667,0.000000,67 days 12:00:00
CHAK,0.500000,0.000000,18 days 00:00:00
FIRN,0.500000,0.000000,23 days 12:00:00
GAN4,0.444444,0.000000,21 days 09:36:00
THUL,0.428571,0.000000,25 days 16:00:00


In [63]:
association_check.nlargest(20, 'length_of_expedition')

,member_deaths,hired_staff_deaths,length_of_expedition
peak_id,,,
DHA5,0.666667,0.000000,67 days 12:00:00
TSUR,0.000000,0.000000,62 days 12:00:00
DZAN,0.000000,0.000000,60 days 00:00:00
LHOM,0.000000,0.000000,60 days 00:00:00
KHUM,0.000000,0.000000,54 days 00:00:00
SNOW,0.000000,0.000000,54 days 00:00:00
MANP,0.000000,0.000000,53 days 00:00:00
DHA4,0.818182,0.454545,52 days 12:00:00
CTSE,0.000000,0.000000,52 days 00:00:00


Still not seeing anything in terms of expedition ~ death, aside from the obvious "it's more difficult to die the less time you spend on the mountain." 